#### Projeto Sistemas de Recomendação
##### Recomendação de filmes baseado em texto de input do usuário

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt

import os
import re
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

2023-10-03 13:54:19.681266: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-03 13:54:19.701545: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-03 13:54:19.701567: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-03 13:54:19.701586: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-03 13:54:19.706194: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-03 13:54:19.706549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
print("module loaded")

2023-10-03 13:54:56.968979: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2023-10-03 13:54:56.983544: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2023-10-03 13:54:56.998242: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2023-10-03 13:54:57.011792: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2023-10-03 13:54:57.025930: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.


module loaded


In [14]:
def embed(input):
    return model(input)

In [10]:
df_movies = pd.read_csv("Top_10000_Movies.csv", delimiter=',', engine='python')
df_movies.head()

,Unnamed: 0,id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
0,0,580489.0,en,Venom: Let There Be Carnage,5401.308,2021-09-30,6.8,1736.0,"['Science Fiction', 'Action', 'Adventure']",After finding a host body in investigative rep...,424000000.0,97.0,NaN
1,1,524434.0,en,Eternals,3365.535,2021-11-03,7.1,622.0,"['Action', 'Adventure', 'Science Fiction', 'Fa...",The Eternals are a team of ancient aliens who ...,165000000.0,157.0,In the beginning...
2,2,438631.0,en,Dune,2911.423,2021-09-15,8.0,3632.0,"['Action', 'Adventure', 'Science Fiction']","Paul Atreides, a brilliant and gifted young ma...",331116356.0,155.0,"Beyond fear, destiny awaits."
3,3,796499.0,en,Army of Thieves,2552.437,2021-10-27,6.9,555.0,"['Action', 'Crime', 'Thriller']",A mysterious woman recruits bank teller Ludwig...,0.0,127.0,"Before Vegas, one locksmith became a legend."
4,4,550988.0,en,Free Guy,1850.470,2021-08-11,7.8,3493.0,"['Comedy', 'Action', 'Adventure', 'Science Fic...",A bank teller called Guy realizes he is a back...,331096766.0,115.0,Life's too short to be a background character.


In [12]:
df_movies_overview = df_movies[['original_title', 'overview']]
df_movies_overview.head()

,original_title,overview
0,Venom: Let There Be Carnage,After finding a host body in investigative rep...
1,Eternals,The Eternals are a team of ancient aliens who ...
2,Dune,"Paul Atreides, a brilliant and gifted young ma..."
3,Army of Thieves,A mysterious woman recruits bank teller Ludwig...
4,Free Guy,A bank teller called Guy realizes he is a back...


In [13]:
df_movies_overview = df_movies_overview.dropna()
df_movies_overview = df_movies_overview.reset_index(drop=True)
df_movies_overview = df_movies_overview[:5500] #Evitar overflow
list_overview = list(df_movies_overview['overview'])
list_overview[:5]


['After finding a host body in investigative reporter Eddie Brock, the alien symbiote must face a new enemy, Carnage, the alter ego of serial killer Cletus Kasady.',
 'The Eternals are a team of ancient aliens who have been living on Earth in secret for thousands of years. When an unexpected tragedy forces them out of the shadows, they are forced to reunite against mankind’s most ancient enemy, the Deviants.',
 "Paul Atreides, a brilliant and gifted young man born into a great destiny beyond his understanding, must travel to the most dangerous planet in the universe to ensure the future of his family and his people. As malevolent forces explode into conflict over the planet's exclusive supply of the most precious resource in existence-a commodity capable of unlocking humanity's greatest potential-only those who can conquer their fear will survive.",
 'A mysterious woman recruits bank teller Ludwig Dieter to lead a group of aspiring thieves on a top-secret heist during the early stages 

In [16]:
embedings = embed(list_overview)
embedings

<tf.Tensor: shape=(5500, 512), dtype=float32, numpy=
array([[ 0.03195498,  0.01191413, -0.0692939 , ...,  0.01460928,
         0.07080539,  0.0210987 ],
       [ 0.00416227, -0.03524458, -0.0082063 , ..., -0.05105983,
         0.03969113,  0.02537928],
       [-0.01211539, -0.06974179, -0.02834297, ..., -0.02725488,
         0.05366101,  0.00415146],
       ...,
       [-0.06440541,  0.02434653,  0.02397839, ..., -0.05156148,
         0.04750916,  0.03391587],
       [-0.0183999 , -0.02036232,  0.01738134, ..., -0.04658483,
         0.07610386,  0.03525955],
       [ 0.05090077,  0.02924388,  0.0009307 , ..., -0.04580587,
         0.07124734, -0.02662512]], dtype=float32)>

In [17]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(embedings)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [18]:
def recommend(input):
    embed_input = embed([input])
    distances, indices = knn.kneighbors(embed_input)
    return df_movies_overview.iloc[indices[0]]

In [23]:
recommend("Car action movie with racing and explosions")

,original_title,overview
782,Cars 2,Star race car Lightning McQueen and his pal Ma...
1727,Point Break,A young undercover FBI agent infiltrates a gan...
820,Tropic Thunder,"Ben Stiller, Jack Black and Robert Downey Jr. ..."
2994,Die Hard: With a Vengeance,New York detective John McClane is back and ki...
3971,Speed Racer,Speed Racer is the tale of a young and brillia...
2847,Death Race: Beyond Anarchy,Black Ops specialist Connor Gibson infiltrates...
3065,Thor: Love and Thunder,The fourth installment of the Marvel Studios' ...
4978,Scooby-Doo! and WWE: Curse of the Speed Demon,When Scooby and Mystery Inc. visit an off-road...
4906,Rules of Engagement,"Part war movie, part courtroom thriller, this ..."
4638,You Only Live Twice,A mysterious spacecraft captures Russian and A...
